In [3]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
%matplotlib inline

np.random.seed(0)
import mltools as ml

In [4]:
X = np.genfromtxt("data/X_train.txt", delimiter = None)
Y = np.genfromtxt("data/Y_train.txt", delimiter = None)
np.random.seed(0)



X, Y = ml.shuffleData(X, Y)
Xtr = X[:10000, :]
Xva = X[10000:20000, :]
Ytr = Y[:10000]
Yva = Y[10000:20000]


In [5]:
class knnCFls(ml.base.classifier):
    def __init__(self, learners):
        """Constructs a BaggedTree class with a set of learners. """
        self.learners = learners
    
    def predictSoft(self, X):
        """Predicts the probabilities with each bagged learner and average over the results. """
        n_bags = len(self.learners)
        preds = [self.learners[l].predictSoft(X) for l in range(n_bags)]
        return np.mean(preds, axis=0)

In [ ]:
def knnCF(X, Y, Xte, K):
    clusterDict = defaultdict(list)
    xCopy = np.copy(X)
    auctr = []
    aucva = []
    learners = []

    Z, d = ml.cluster.agglomerative(xCopy, K=50, method='max')
    for i in range(99):
        if Z[i] not in clusterDict:
            clusterDict[Z[i]] = [(xCopy[i], Y[i])]
        else:
            clusterDict[Z[i]].append((xCopy[i], Y[i]))
     
    for item in clusterDict.items():
        xtr = []
        ytr = []
        tmse = []
        vmse = []

        for tr in range(len(item[1])):
            xtr.append(item[1][tr][0])
            ytr.append(item[1][tr][1])
        xtr = np.array(xtr)
        ytr = np.array(ytr)


        learner = ml.knn.knnClassify(xtr, ytr, K)
        learners.append(learner)
        
#         if len(xtr) != 1 and np.sum(ytr) != len(ytr) and np.sum(ytr) != 0:
#             auctr.append(learner.auc(xtr, ytr))
#             aucva.append(learner.auc(Xva, Yva))
#         else:
#             auctr.append(ytr[0])
#             aucva.append(Ytr[0])
#     auctr = np.array(auctr)
    aucva = np.array(aucva)
    knnls = knnCFls(learners)
#     trResult = np.mean(auctr)
#     vaResult = np.mean(aucva)
    
    Yte = np.vstack((np.arange(Xte.shape[0]), knnls.predictSoft(Xte)[:,1])).T
    # Output a file with two columns, a row ID and a confidence in class 1:
    np.savetxt('Y_submit_knn.txt',Yte,'%d, %.2f',header='ID,Prob1',comments='',delimiter=',')
    
#     print "trResult", trResult
#     print "vaResult", vaResult             
#     print tmse
#     print vmse
    
    return

In [ ]:
Xte = np.genfromtxt('data/X_test.txt', delimiter=None)
knnCF(X, Y, Xte, 20)
